In [1]:
from google.colab import drive
drive.mount('/content/drive')

!git clone https://github.com/neccam/slt.git

%cd slt

!chmod +x data/download.sh
!data/download.sh

!cp ../drive/MyDrive/slt/requirements.txt requirements.txt

!pip -q install -r requirements.txt

Mounted at /content/drive
Cloning into 'slt'...
remote: Enumerating objects: 99, done.
remote: Total 99 (delta 0), reused 0 (delta 0), pack-reused 99
Unpacking objects: 100% (99/99), done.
/content/slt
--2021-01-04 17:11:18--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.train
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1935289362 (1.8G)
Saving to: ‘phoenix14t.pami0.train’

phoenix14t.pami0.tr 100%[===================>]   1.80G  15.1MB/s    in 94s     

2021-01-04 17:12:53 (19.7 MB/s) - ‘phoenix14t.pami0.train’ saved [1935289362/1935289362]

--2021-01-04 17:12:53--  http://cihancamgoz.com/files/cvpr2020/phoenix14t.pami0.dev
Resolving cihancamgoz.com (cihancamgoz.com)... 192.232.218.145
Connecting to cihancamgoz.com (cihancamgoz.com)|192.232.218.145|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13

In [2]:
!cp ../drive/MyDrive/slt/late-fusion-duo/*.py signjoey/

In [3]:
!mkdir data/PHOENIX2014T 

!mv phoenix14t.pami0.dev data/PHOENIX2014T 
!mv phoenix14t.pami0.test data/PHOENIX2014T 
!mv phoenix14t.pami0.train data/PHOENIX2014T

!unzip -q '../drive/MyDrive/slt/keypoints/train_body_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_face_pose3d.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/train_hand_pose3d.zip' -d data

!unzip -q '../drive/MyDrive/slt/keypoints/test_keypoints.zip' -d data
!unzip -q '../drive/MyDrive/slt/keypoints/dev_keypoints.zip' -d data

In [4]:
from six.moves import cPickle
import gzip
import torch
import os
import numpy as np
from tqdm import tqdm
import pickle

wd='data'

def load_dataset_file(filename):
    with gzip.open(filename, "rb") as f:
        loaded_object = pickle.load(f)
        return loaded_object

## Keypoints

In [5]:
body_parts = ['body_pose3d',
              'face_pose3d',
              'hand_pose3d']

for s in ['dev', 'test', 'train']:
  filename = "data/PHOENIX2014T/phoenix14t.pami0.{}".format(s)
  src_features = load_dataset_file(filename)
  dst_filename = "data/PHOENIX2014T/phoenix14t.pami1.{}".format(s)
  for src in tqdm(src_features):
    name = os.path.basename(src['name']) + '.npy'
    
    new_features_list = []

    for body_part in body_parts:
      dst = os.path.join(os.path.join(wd, s), os.path.join(body_part, name))

      new_features = np.load(dst)
      new_features = new_features.reshape((new_features.shape[0], np.prod(new_features.shape[1: ])))
      new_features_list.append(new_features)
      
    
    new_features = np.concatenate(new_features_list, axis=1)
    src['sign'] = torch.Tensor(new_features)

  with gzip.open(dst_filename, "wb") as f:
    pickle.dump(src_features, f)

100%|██████████| 7096/7096 [00:15<00:00, 468.75it/s]


## Change config files

In [6]:
import yaml
with open("configs/sign.yaml") as f:
     list_doc = yaml.load(f)

for s in ['train', 'test', 'dev']:
  list_doc['data'][s] = ['PHOENIX2014T/phoenix14t.pami{}.{}'.format(i, s) for i in range(2)]

list_doc['data']['feature_size'] = [1024, 417]

with open("configs/sign.yaml", "w") as f:
    yaml.dump(list_doc, f)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  This is separate from the ipykernel package so we can avoid doing imports until


## Run

In [7]:
!cp -r '../drive/MyDrive/slt/models/baseline_20' baseline
!cp -r '../drive/MyDrive/slt/models/keypoints3d_20' keypoints3d

In [8]:
%cd slt

[Errno 2] No such file or directory: 'slt'
/content/slt


In [9]:
import os
import sys
from signjoey.prediction import test

test(cfg_file="configs/sign.yaml", ckpts=["baseline/best.ckpt", "keypoints3d/best.ckpt"], output_path="")

2021-01-04 17:21:49,466 - ------------------------------------------------------------
2021-01-04 17:21:49,467 - [DEV] partition [RECOGNITION] experiment [BW]: 1
/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:359: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  var = torch.tensor(arr, dtype=self.dtype, device=device)
2021-01-04 17:22:18,014 - finished in 28.5475s 
2021-01-04 17:22:18,016 - ************************************************************
2021-01-04 17:22:18,017 - [DEV] partition [RECOGNITION] results:
	New Best CTC Decode Beam Size: 1
	WER 52.12	(DEL: 47.96,	INS: 0.48,	SUB: 3.68)
2021-01-04 17:22:18,017 - ************************************************************
2021-01-04 17:22:18,018 - ------------------------------------------------------------
2021-01-04 17:22:18,018 - [DEV] partition [RECOGNITION] exper

translation


2021-01-04 17:39:26,831 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 1 and Alpha: -1
	BLEU-4 18.28	(BLEU-1: 44.54,	BLEU-2: 30.56,	BLEU-3: 22.95,	BLEU-4: 18.28)
	CHRF 41.44	ROUGE 44.59
2021-01-04 17:39:26,832 - ------------------------------------------------------------
2021-01-04 17:42:38,042 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: -1
	BLEU-4 18.29	(BLEU-1: 44.11,	BLEU-2: 30.68,	BLEU-3: 23.11,	BLEU-4: 18.29)
	CHRF 41.36	ROUGE 45.66
2021-01-04 17:42:38,043 - ------------------------------------------------------------
2021-01-04 17:43:34,197 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 and Alpha: 1
	BLEU-4 18.76	(BLEU-1: 44.56,	BLEU-2: 31.13,	BLEU-3: 23.57,	BLEU-4: 18.76)
	CHRF 41.83	ROUGE 45.70
2021-01-04 17:43:34,198 - ------------------------------------------------------------
2021-01-04 17:44:02,370 - [DEV] partition [Translation] results:
	New Best Translation Beam Size: 2 a